# **Chapter 5 – Support Vector Machines**

_This notebook contains all the sample code and solutions to the exercises in chapter 5._

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20.

In [ ]:
# ============================================
# CONFIGURACIÓN INICIAL DEL NOTEBOOK
# ============================================

# Importamos las bibliotecas fundamentales para el análisis
import sys
import sklearn  # Biblioteca de machine learning
import numpy as np  # Manejo de arrays y operaciones numéricas
import os

# Establecemos una semilla aleatoria para reproducibilidad de resultados
np.random.seed(42)

# ============================================
# CONFIGURACIÓN DE MATPLOTLIB PARA GRÁFICOS
# ============================================
# Habilitamos el modo inline para que los gráficos se muestren en el notebook
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Configuramos el tamaño de fuente para mejorar la legibilidad de los gráficos
mpl.rc('axes', labelsize=14)  # Tamaño de etiquetas de los ejes
mpl.rc('xtick', labelsize=12)  # Tamaño de las marcas del eje X
mpl.rc('ytick', labelsize=12)  # Tamaño de las marcas del eje Y

# Large margin classification

The next few code cells generate the first figures in chapter 5. The first actual code sample comes after:

In [ ]:
# ============================================
# CARGA DEL DATASET IRIS Y PREPARACIÓN
# ============================================

from sklearn.svm import SVC  # Support Vector Classifier
from sklearn import datasets

# Cargamos el famoso dataset Iris
iris = datasets.load_iris()

# Seleccionamos solo dos características para facilitar la visualización 2D:
# - Columna 2: Longitud del pétalo (petal length)
# - Columna 3: Ancho del pétalo (petal width)
X = iris["data"][:, (2, 3)]
y = iris["target"]  # Las etiquetas de clase (0=setosa, 1=versicolor, 2=virginica)

# ============================================
# FILTRADO DE CLASES PARA CLASIFICACIÓN BINARIA
# ============================================
# SVM funciona mejor con problemas binarios, así que filtramos solo dos clases:
# - Clase 0: Iris Setosa
# - Clase 1: Iris Versicolor
setosa_or_versicolor = (y == 0) | (y == 1)
X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

# ============================================
# CREACIÓN DEL MODELO SVM CON KERNEL LINEAL
# ============================================
# Creamos un clasificador SVM con las siguientes características:
# - kernel='linear': Usamos un kernel lineal (frontera de decisión recta)
# - C = 1*10**10: Un valor MUY alto de C que penaliza fuertemente los errores
#   (esto crea un "hard margin" - margen duro sin violaciones)
svm_clf = SVC(kernel='linear', C = 1*10**10)

In [ ]:
# ============================================
# ENTRENAMIENTO DEL MODELO SVM
# ============================================
# Ajustamos (entrenamos) el modelo SVM con nuestros datos
# El modelo aprenderá a encontrar el hiperplano óptimo que separa las dos clases
# maximizando el margen entre ellas
svm_clf.fit(X, y)

In [ ]:
# ============================================
# CREACIÓN DE MODELOS "MALOS" PARA COMPARACIÓN
# ============================================
# Creamos algunos modelos de clasificación subóptimos para demostrar
# por qué SVM es superior (encuentra el mejor margen)
x0 = np.linspace(0, 5.5, 200)
pred_1 = 5*x0 - 20    # Primera frontera de decisión candidata
pred_2 = x0 - 1.8     # Segunda frontera de decisión candidata
pred_3 = 0.1 * x0 + 0.5  # Tercera frontera de decisión candidata

# ============================================
# FUNCIÓN PARA VISUALIZAR LA FRONTERA DE DECISIÓN SVM
# ============================================
def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    """
    Esta función dibuja la frontera de decisión del SVM y sus márgenes.
    
    Parámetros:
    - svm_clf: modelo SVM entrenado
    - xmin, xmax: rango del eje x para la visualización
    """
    # Extraemos los coeficientes del hiperplano (w0, w1) y el intercepto (b)
    w = svm_clf.coef_[0]      # Vector de pesos [w0, w1]
    b = svm_clf.intercept_[0]  # Término independiente (bias)

    # La frontera de decisión es: w0*x0 + w1*x1 + b = 0
    # Despejamos x1: x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]

    # Calculamos los márgenes (las líneas paralelas a la frontera)
    # El margen tiene un ancho de 2/||w||
    margin = 1/w[1]
    gutter_up = decision_boundary + margin    # Margen superior
    gutter_down = decision_boundary - margin  # Margen inferior

    # Destacamos los vectores de soporte (support vectors)
    # Son los puntos más cercanos a la frontera de decisión
    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='#FFAAAA')
    
    # Dibujamos la frontera de decisión (línea sólida) y los márgenes (líneas punteadas)
    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, gutter_up, "k--", linewidth=2)
    plt.plot(x0, gutter_down, "k--", linewidth=2)

# ============================================
# CREACIÓN DE GRÁFICOS COMPARATIVOS
# ============================================
plt.figure(figsize=(12,2.7))

# GRÁFICO IZQUIERDO: Múltiples fronteras de decisión posibles (modelos "malos")
plt.subplot(121)
plt.plot(x0, pred_1, "g--", linewidth=2)  # Frontera candidata 1
plt.plot(x0, pred_2, "m-", linewidth=2)   # Frontera candidata 2
plt.plot(x0, pred_3, "r-", linewidth=2)   # Frontera candidata 3
# Graficamos los puntos de datos
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.axis([0, 5.5, 0, 2])

# GRÁFICO DERECHO: Frontera de decisión óptima encontrada por SVM
plt.subplot(122)
plot_svc_decision_boundary(svm_clf, 0, 5.5)
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo")
plt.xlabel("Petal length", fontsize=14)
plt.axis([0, 5.5, 0, 2]);

# Sensitivity to feature scales

In [ ]:
# ============================================
# DEMOSTRACIÓN: SENSIBILIDAD A ESCALAS DE CARACTERÍSTICAS
# ============================================
# Este ejemplo muestra por qué es CRÍTICO escalar las características antes de usar SVM

# Creamos un dataset sintético con escalas muy diferentes:
# - x0: valores entre 1 y 5
# - x1: valores entre 20 y 80 (mucho más grandes)
Xs = np.array([[1, 50], [5, 20], [3, 80], [5, 60]]).astype(np.float64)
ys = np.array([0, 0, 1, 1])  # Etiquetas de clase

# Entrenamos un SVM con datos SIN ESCALAR
svm_clf = SVC(kernel="linear", C=100)
svm_clf.fit(Xs, ys)

# ============================================
# GRÁFICO COMPARATIVO: ANTES Y DESPUÉS DEL ESCALADO
# ============================================
plt.figure(figsize=(12,3.2))

# GRÁFICO IZQUIERDO: Datos sin escalar
plt.subplot(121)
plt.plot(Xs[:, 0][ys==1], Xs[:, 1][ys==1], "bo")  # Clase 1
plt.plot(Xs[:, 0][ys==0], Xs[:, 1][ys==0], "ms")  # Clase 0
plot_svc_decision_boundary(svm_clf, 0, 6)
plt.xlabel("$x_0$", fontsize=20)
plt.ylabel("$x_1$  ", fontsize=20, rotation=0)
plt.title("Unscaled", fontsize=16)
plt.axis([0, 6, 0, 90])
# NOTA: La frontera de decisión se ve casi vertical porque x1 domina (escala mayor)

# Aplicamos StandardScaler para normalizar las características
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(Xs)  # Transforma los datos: (x - media) / desv_std

# Entrenamos un nuevo SVM con datos ESCALADOS
svm_clf.fit(X_scaled, ys)

# GRÁFICO DERECHO: Datos escalados (StandardScaler)
plt.subplot(122)
plt.plot(X_scaled[:, 0][ys==1], X_scaled[:, 1][ys==1], "bo")  # Clase 1
plt.plot(X_scaled[:, 0][ys==0], X_scaled[:, 1][ys==0], "ms")  # Clase 0
plot_svc_decision_boundary(svm_clf, -2, 2)
plt.xlabel("$x_0$", fontsize=20)
plt.title("Scaled", fontsize=16)
plt.axis([-2, 2, -2, 2])
# NOTA: Ahora la frontera de decisión tiene una orientación más equilibrada
# porque ambas características tienen la misma escala (media=0, std=1)

# Sensitivity to outliers

In [ ]:
# ============================================
# DEMOSTRACIÓN: SENSIBILIDAD A OUTLIERS (VALORES ATÍPICOS)
# ============================================
# Este ejemplo muestra cómo los outliers pueden afectar la frontera de decisión del SVM

# Creamos dos outliers (puntos atípicos) de clase 0 en una zona dominada por clase 1
X_outliers = np.array([[3.4, 1.3], [3.2, 0.8]])
y_outliers = np.array([0, 0])

# Creamos dos datasets diferentes añadiendo outliers:
# Dataset 1: datos originales + primer outlier
Xo1 = np.concatenate([X, X_outliers[:1]], axis=0)
yo1 = np.concatenate([y, y_outliers[:1]], axis=0)

# Dataset 2: datos originales + segundo outlier
Xo2 = np.concatenate([X, X_outliers[1:]], axis=0)
yo2 = np.concatenate([y, y_outliers[1:]], axis=0)

# Entrenamos un SVM con C muy alto (hard margin - margen duro)
# Esto hace que el modelo sea MUY sensible a outliers
svm_clf2 = SVC(kernel="linear", C=10**9)
svm_clf2.fit(Xo2, yo2)

# ============================================
# GRÁFICOS COMPARATIVOS
# ============================================
plt.figure(figsize=(12,2.7))

# GRÁFICO IZQUIERDO: Outlier que hace IMPOSIBLE la separación lineal perfecta
plt.subplot(121)
plt.plot(Xo1[:, 0][yo1==1], Xo1[:, 1][yo1==1], "bs")  # Clase Versicolor
plt.plot(Xo1[:, 0][yo1==0], Xo1[:, 1][yo1==0], "yo")  # Clase Setosa
plt.text(0.3, 1.0, "Impossible!", fontsize=24, color="red")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
# Anotamos dónde está el outlier problemático
plt.annotate("Outlier",
             xy=(X_outliers[0][0], X_outliers[0][1]),
             xytext=(2.5, 1.7),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.1),
             fontsize=16,
            )
plt.axis([0, 5.5, 0, 2])

# GRÁFICO DERECHO: Outlier que DISTORSIONA la frontera de decisión
plt.subplot(122)
plt.plot(Xo2[:, 0][yo2==1], Xo2[:, 1][yo2==1], "bs")  # Clase Versicolor
plt.plot(Xo2[:, 0][yo2==0], Xo2[:, 1][yo2==0], "yo")  # Clase Setosa
plot_svc_decision_boundary(svm_clf2, 0, 5.5)
plt.xlabel("Petal length", fontsize=14)
# Anotamos dónde está el outlier que distorsiona el modelo
plt.annotate("Outlier",
             xy=(X_outliers[1][0], X_outliers[1][1]),
             xytext=(3.2, 0.08),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.1),
             fontsize=16,
            )
plt.axis([0, 5.5, 0, 2])
# NOTA: La frontera de decisión se desvía para intentar clasificar correctamente el outlier
# Esto puede llevar a un modelo con mala generalización (overfitting)

# Large margin *vs* margin violations

This is the first code example in chapter 5:

In [ ]:
# ============================================
# EJEMPLO PRÁCTICO: PIPELINE COMPLETO CON SVM
# ============================================
# Este es el primer ejemplo de código completo del capítulo 5

import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC  # Versión optimizada para SVM lineal

# Cargamos el dataset Iris completo
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width

# Creamos un problema de clasificación binaria:
# ¿Es la flor Iris virginica (clase 2) o no?
y = (iris["target"] == 2).astype(np.float64)  # 1 si es virginica, 0 si no

# ============================================
# CREACIÓN DE UN PIPELINE CON MEJORES PRÁCTICAS
# ============================================
# Un Pipeline encadena múltiples pasos de procesamiento:
svm_clf = Pipeline([
    # Paso 1: Escalado de características (CRÍTICO para SVM)
    ("scaler", StandardScaler()),
    
    # Paso 2: Clasificador SVM lineal
    # - C = 1: Parámetro de regularización (equilibrio entre margen y violaciones)
    # - loss='hinge': Función de pérdida estándar para SVM
    # - random_state=42: Para reproducibilidad
    ("linear_svc", LinearSVC(C = 1, loss='hinge', random_state=42))
])

# Entrenamos el pipeline completo (escala + entrena)
svm_clf.fit(X, y)
# NOTA: El Pipeline automáticamente aplica el escalado y luego entrena el SVM

Now let's generate the graph comparing different regularization settings:

In [8]:
np.unique(y)

array([0., 1.])

In [ ]:
# ============================================
# COMPARACIÓN DEL PARÁMETRO C (REGULARIZACIÓN)
# ============================================
# El parámetro C controla el trade-off entre:
# - Margen amplio (mejor generalización)
# - Menos violaciones del margen (mejor precisión en entrenamiento)

scaler = StandardScaler()

# ============================================
# MODELO 1: C = 1 (Regularización FUERTE)
# ============================================
# C bajo = Permite más violaciones del margen
# Resultado: Margen MÁS AMPLIO pero menos preciso en los datos de entrenamiento
svm_clf1 = LinearSVC(C=1, loss="hinge", random_state=42, max_iter = 10000)

# ============================================
# MODELO 2: C = 100 (Regularización DÉBIL)
# ============================================
# C alto = Penaliza fuertemente las violaciones del margen
# Resultado: Margen MÁS ESTRECHO pero más preciso en los datos de entrenamiento
svm_clf2 = LinearSVC(C=100, loss="hinge", random_state=42, max_iter = 10000)

# Creamos pipelines para ambos modelos
scaled_svm_clf1 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf1),
    ])

scaled_svm_clf2 = Pipeline([
        ("scaler", scaler),
        ("linear_svc", svm_clf2),
    ])

# Entrenamos ambos modelos con los mismos datos
scaled_svm_clf1.fit(X, y)
scaled_svm_clf2.fit(X, y)
# NOTA: Veremos cómo C afecta la frontera de decisión y el ancho del margen

In [ ]:
# ============================================
# CONVERSIÓN A PARÁMETROS SIN ESCALAR
# ============================================
# Para visualizar correctamente, necesitamos convertir los parámetros del modelo
# (que fueron entrenados con datos escalados) de vuelta a la escala original

# Calculamos el bias (intercepto) en la escala original
b1 = svm_clf1.decision_function([-scaler.mean_ / scaler.scale_])
b2 = svm_clf2.decision_function([-scaler.mean_ / scaler.scale_])

# Calculamos los pesos (coeficientes) en la escala original
# Dividimos por scale_ para "deshacer" el escalado
w1 = svm_clf1.coef_[0] / scaler.scale_
w2 = svm_clf2.coef_[0] / scaler.scale_

# Asignamos los parámetros convertidos de vuelta a los modelos
svm_clf1.intercept_ = np.array([b1])
svm_clf2.intercept_ = np.array([b2])
svm_clf1.coef_ = np.array([w1])
svm_clf2.coef_ = np.array([w2])

# ============================================
# IDENTIFICACIÓN MANUAL DE VECTORES DE SOPORTE
# ============================================
# LinearSVC no identifica automáticamente los vectores de soporte,
# así que los encontramos manualmente

# Convertimos las etiquetas a {-1, +1} (formato estándar de SVM)
t = y * 2 - 1

# Los vectores de soporte son los puntos que:
# - Están en el margen: t * (X·w + b) = 1
# - Violan el margen: t * (X·w + b) < 1
# Aquí buscamos todos los puntos con t * (X·w + b) < 1
support_vectors_idx1 = (t * (X.dot(w1) + b1) < 1).ravel()
support_vectors_idx2 = (t * (X.dot(w2) + b2) < 1).ravel()

# Guardamos los vectores de soporte en los modelos
svm_clf1.support_vectors_ = X[support_vectors_idx1]
svm_clf2.support_vectors_ = X[support_vectors_idx2]

In [ ]:
# ============================================
# VISUALIZACIÓN: COMPARACIÓN DE C=1 vs C=100
# ============================================
# Este gráfico muestra el impacto del parámetro C en la frontera de decisión

plt.figure(figsize=(12,3.2))

# GRÁFICO IZQUIERDO: C = 1 (Regularización FUERTE)
plt.subplot(121)
# Dibujamos los puntos de datos
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^", label="Iris virginica")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs", label="Iris versicolor")
# Dibujamos la frontera de decisión
plot_svc_decision_boundary(svm_clf1, 4, 6)
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper left", fontsize=14)
plt.title("$C = {}$".format(svm_clf1.C), fontsize=16)
plt.axis([4, 6, 0.8, 2.8])
# NOTA: Con C=1 vemos un margen MÁS AMPLIO (líneas punteadas más separadas)
# Algunos puntos pueden estar dentro del margen (violaciones permitidas)

# GRÁFICO DERECHO: C = 100 (Regularización DÉBIL)
plt.subplot(122)
# Dibujamos los puntos de datos
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")
# Dibujamos la frontera de decisión
plot_svc_decision_boundary(svm_clf2, 4, 6)
plt.xlabel("Petal length", fontsize=14)
plt.title("$C = {}$".format(svm_clf2.C), fontsize=16)
plt.axis([4, 6, 0.8, 2.8])
# NOTA: Con C=100 vemos un margen MÁS ESTRECHO (líneas punteadas más juntas)
# El modelo penaliza más las violaciones, ajustándose más a los datos de entrenamiento
# ¡Cuidado con el overfitting!

# Non-linear classification

In [ ]:
# ============================================
# INTRODUCCIÓN A CLASIFICACIÓN NO LINEAL
# ============================================
# A veces los datos NO son linealmente separables en el espacio original
# Solución: Transformarlos a un espacio de mayor dimensión

# Creamos un dataset 1D simple (no separable linealmente)
X1D = np.linspace(-4, 4, 9).reshape(-1, 1)
# Añadimos una característica polinómica: x2 = x1²
X2D = np.c_[X1D, X1D**2]  # Ahora tenemos [x1, x1²]
y = np.array([0, 0, 1, 1, 1, 1, 1, 0, 0])

plt.figure(figsize=(11, 4))

# ============================================
# GRÁFICO IZQUIERDO: Espacio original 1D (NO separable linealmente)
# ============================================
plt.subplot(121)
plt.grid(True, which='both')
plt.axhline(y=0, color='k')  # Línea horizontal en y=0
# Dibujamos los puntos sobre el eje x
plt.plot(X1D[:, 0][y==0], np.zeros(4), "bs")  # Clase 0 en azul
plt.plot(X1D[:, 0][y==1], np.zeros(5), "g^")  # Clase 1 en verde
plt.gca().get_yaxis().set_ticks([])
plt.xlabel(r"$x_1$", fontsize=20)
plt.axis([-4.5, 4.5, -0.2, 0.2])
# NOTA: En 1D, no hay una línea recta que separe perfectamente las dos clases

# ============================================
# GRÁFICO DERECHO: Espacio transformado 2D (SÍ separable linealmente)
# ============================================
plt.subplot(122)
plt.grid(True, which='both')
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
# Dibujamos los puntos en el espacio 2D [x1, x1²]
plt.plot(X2D[:, 0][y==0], X2D[:, 1][y==0], "bs")  # Clase 0
plt.plot(X2D[:, 0][y==1], X2D[:, 1][y==1], "g^")  # Clase 1
plt.xlabel(r"$x_1$", fontsize=20)
plt.ylabel(r"$x_2$", fontsize=20, rotation=0)
plt.gca().get_yaxis().set_ticks([0, 4, 8, 12, 16])
# Dibujamos una línea horizontal que separa las clases
plt.plot([-4.5, 4.5], [6.5, 6.5], "r--", linewidth=3)
plt.axis([-4.5, 4.5, -1, 17])
# NOTA: En 2D (con la característica x1²), las clases SÍ son linealmente separables
# Esta es la idea clave del "kernel trick" en SVM

plt.subplots_adjust(right=1);

In [ ]:
# ============================================
# CREACIÓN DE UN DATASET NO LINEAL: "MOONS"
# ============================================
# make_moons genera un dataset con forma de dos lunas entrelazadas
# Este dataset NO es linealmente separable

from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

# Generamos 100 muestras con algo de ruido
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

# Función auxiliar para visualizar el dataset
def plot_dataset(X, y, axes):
    """
    Dibuja un dataset 2D con dos clases
    """
    plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bs")  # Clase 0 en cuadrados azules
    plt.plot(X[:, 0][y==1], X[:, 1][y==1], "g^")  # Clase 1 en triángulos verdes
    plt.axis(axes)
    plt.grid(True, which='both')
    plt.xlabel(r"$x_1$", fontsize=20)
    plt.ylabel(r"$x_2$", fontsize=20, rotation=0)

# Visualizamos el dataset
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
# NOTA: Las dos clases tienen forma de "lunas" y están entrelazadas
# No es posible separarlas con una línea recta (SVM lineal no funcionará bien)

In [ ]:
# ============================================
# ENFOQUE 1: CARACTERÍSTICAS POLINÓMICAS + SVM LINEAL
# ============================================
# Una forma de manejar datos no lineales es:
# 1. Añadir características polinómicas (x², x³, x1*x2, etc.)
# 2. Aplicar SVM lineal en el espacio de características ampliado

from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_svm_clf = Pipeline([
        # Paso 1: Crear características polinómicas hasta grado 3
        # Por ejemplo, de [x1, x2] crea: [1, x1, x2, x1², x1*x2, x2², x1³, ...]
        ("poly_features", PolynomialFeatures(degree=3)),
        
        # Paso 2: Escalar las características (CRÍTICO)
        ("scaler", StandardScaler()),
        
        # Paso 3: Aplicar SVM lineal en el espacio de características ampliado
        ("svm_clf", LinearSVC(C=10, loss="hinge", max_iter = 10000, random_state=42))
    ])

# Entrenamos el pipeline completo
polynomial_svm_clf.fit(X, y)
# NOTA: Aunque usamos LinearSVC, la frontera de decisión será NO LINEAL
# en el espacio original porque trabajamos con características polinómicas

In [ ]:
# ============================================
# FUNCIÓN PARA VISUALIZAR PREDICCIONES DEL MODELO
# ============================================
def plot_predictions(clf, axes):
    """
    Dibuja las regiones de decisión y los contornos de un clasificador
    
    Parámetros:
    - clf: clasificador entrenado
    - axes: límites del gráfico [xmin, xmax, ymin, ymax]
    """
    # Creamos una malla de puntos para evaluar el modelo
    x0s = np.linspace(axes[0], axes[1], 100)  # 100 puntos en el eje x
    x1s = np.linspace(axes[2], axes[3], 100)  # 100 puntos en el eje y
    x0, x1 = np.meshgrid(x0s, x1s)  # Creamos la malla 2D
    
    # Convertimos la malla a formato de array (cada fila es un punto [x0, x1])
    X = np.c_[x0.ravel(), x1.ravel()]
    
    # Predecimos la clase para cada punto de la malla
    y_pred = clf.predict(X).reshape(x0.shape)
    
    # Calculamos el valor de la función de decisión (distancia al hiperplano)
    y_decision = clf.decision_function(X).reshape(x0.shape)
    
    # Dibujamos las regiones de decisión con colores de fondo
    plt.contourf(x0, x1, y_pred, cmap=plt.cm.brg, alpha=0.2)
    
    # Dibujamos los contornos de la función de decisión
    plt.contourf(x0, x1, y_decision, cmap=plt.cm.brg, alpha=0.1)

# ============================================
# VISUALIZACIÓN DEL RESULTADO
# ============================================
# Dibujamos las regiones de decisión del modelo
plot_predictions(polynomial_svm_clf, [-1.5, 2.5, -1, 1.5])
# Superponemos los puntos de datos originales
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5]);
# NOTA: Observa cómo la frontera de decisión es CURVA (no lineal)
# a pesar de usar LinearSVC internamente

In [ ]:
# ============================================
# ENFOQUE 2: KERNEL POLINÓMICO (KERNEL TRICK)
# ============================================
# En lugar de crear características polinómicas explícitamente,
# usamos el "kernel trick" que es MUCHO más eficiente

from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        # Paso 1: Escalar las características
        ("scaler", StandardScaler()),
        
        # Paso 2: SVM con kernel polinómico
        # - kernel="poly": Usa el kernel polinómico (no necesita PolynomialFeatures)
        # - degree=3: Grado del polinomio
        # - coef0=1: Término independiente (controla la influencia de términos de alto grado)
        # - C=5: Parámetro de regularización
        ("svm_clf", SVC(kernel = "poly", degree=3, coef0=1, C=5))
    ])

# Entrenamos el modelo
poly_kernel_svm_clf.fit(X, y)
# NOTA: El "kernel trick" calcula productos escalares en el espacio de alta dimensión
# SIN crear explícitamente las características polinómicas
# ¡Esto ahorra MUCHA memoria y tiempo de cómputo!

In [ ]:
# ============================================
# DEMOSTRACIÓN: OVERFITTING CON HIPERPARÁMETROS EXTREMOS
# ============================================
# Creamos un modelo con hiperparámetros muy altos para mostrar overfitting

poly100_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        
        # SVM con parámetros extremos:
        # - degree=10: Grado DEMASIADO ALTO del polinomio
        # - coef0=100: Término independiente MUY ALTO
        # - C=5: Misma regularización que antes
        ("svm_clf", SVC(kernel = "poly", degree=10, coef0=100, C=5))
    ])

# Entrenamos el modelo con parámetros extremos
poly100_kernel_svm_clf.fit(X, y)
# NOTA: Este modelo probablemente tendrá OVERFITTING
# Se ajustará demasiado a los datos de entrenamiento y generalizará mal

In [ ]:
# ============================================
# COMPARACIÓN VISUAL: BUENOS vs MALOS HIPERPARÁMETROS
# ============================================

plt.figure(figsize=(11, 4))

# GRÁFICO IZQUIERDO: Hiperparámetros RAZONABLES (d=3, r=1, C=5)
plt.subplot(121)
plot_predictions(poly_kernel_svm_clf, [-1.5, 2.5, -1, 1.5])
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.title(r"$d=3, r=1, C=5$", fontsize=18)
# NOTA: La frontera de decisión es suave y sigue la forma general de los datos
# Este modelo probablemente generalizará bien a datos nuevos

# GRÁFICO DERECHO: Hiperparámetros EXTREMOS (d=10, r=100, C=5)
plt.subplot(122)
plot_predictions(poly100_kernel_svm_clf, [-1.5, 2.5, -1, 1.5])
plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
plt.title(r"$d=10, r=100, C=5$", fontsize=18)
# NOTA: La frontera de decisión es MUY irregular y compleja
# Este modelo tiene OVERFITTING - se ajusta demasiado al ruido en los datos
# Probablemente tendrá mal rendimiento con datos nuevos
# 
# LECCIÓN: Es importante elegir hiperparámetros adecuados
# mediante validación cruzada o búsqueda de hiperparámetros

# Regression


In [ ]:
# ============================================
# REGRESIÓN CON SVM (SUPPORT VECTOR REGRESSION - SVR)
# ============================================
# SVM no solo sirve para clasificación, también para regresión

# Configuramos la semilla aleatoria para reproducibilidad
np.random.seed(42)

# Generamos un dataset sintético para regresión
m = 50  # 50 muestras
X = 2 * np.random.rand(m, 1)  # Valores aleatorios entre 0 y 2

# Generamos la variable objetivo con una relación lineal + ruido
# y = 4 + 3*X + ruido
y = (4 + 3 * X + np.random.randn(m, 1)).ravel()
# NOTA: .ravel() convierte el array 2D en 1D (requerido por algunos estimadores)

In [ ]:
# ============================================
# CREACIÓN Y ENTRENAMIENTO DE UN MODELO SVR LINEAL
# ============================================

from sklearn.svm import LinearSVR

# Creamos un modelo de regresión SVM lineal
# - epsilon=1.5: Define el ancho del "tubo" de tolerancia
#   Los puntos dentro del tubo (error < epsilon) NO afectan al modelo
#   Solo los puntos FUERA del tubo son penalizados
# - random_state=42: Para reproducibilidad
svm_reg = LinearSVR(epsilon=1.5, random_state=42)

# Entrenamos el modelo
svm_reg.fit(X, y)
# NOTA: A diferencia de la regresión lineal tradicional que minimiza el error cuadrático,
# SVR busca una línea donde la mayoría de puntos estén dentro de un "tubo" de ancho epsilon

In [ ]:
# ============================================
# COMPARACIÓN DE DIFERENTES VALORES DE EPSILON
# ============================================
# Creamos dos modelos SVR con diferentes anchos de tubo

# MODELO 1: epsilon = 1.5 (tubo ANCHO)
svm_reg1 = LinearSVR(epsilon=1.5, random_state=42)
svm_reg1.fit(X, y)

# MODELO 2: epsilon = 0.5 (tubo ESTRECHO)
svm_reg2 = LinearSVR(epsilon=0.5, random_state=42)
svm_reg2.fit(X, y)

# ============================================
# IDENTIFICACIÓN DE VECTORES DE SOPORTE
# ============================================
# En SVR, los vectores de soporte son los puntos FUERA del tubo epsilon
def find_support_vectors(svm_reg, X, y):
    """
    Encuentra los vectores de soporte para un modelo SVR
    Son los puntos cuyo error es >= epsilon
    """
    y_pred = svm_reg.predict(X)  # Predicciones del modelo
    # Los vectores de soporte están FUERA del tubo (|error| >= epsilon)
    off_margin = (np.abs(y - y_pred) >= svm_reg.epsilon)
    return np.argwhere(off_margin)

# Identificamos los vectores de soporte para ambos modelos
svm_reg1.support_ = find_support_vectors(svm_reg1, X, y)
svm_reg2.support_ = find_support_vectors(svm_reg2, X, y)

# Variables auxiliares para la visualización
eps_x1 = 1
eps_y_pred = svm_reg1.predict([[eps_x1]])
# NOTA: Con epsilon grande, MENOS puntos serán vectores de soporte
# Con epsilon pequeño, MÁS puntos serán vectores de soporte

In [ ]:
# ============================================
# FUNCIÓN PARA VISUALIZAR REGRESIÓN SVM
# ============================================
def plot_svm_regression(svm_reg, X, y, axes):
    """
    Dibuja la línea de regresión y el tubo epsilon de un modelo SVR
    
    Parámetros:
    - svm_reg: modelo SVR entrenado
    - X, y: datos de entrenamiento
    - axes: límites del gráfico [xmin, xmax, ymin, ymax]
    """
    # Creamos puntos para graficar la línea de regresión
    x1s = np.linspace(axes[0], axes[1], 100).reshape(100, 1)
    y_pred = svm_reg.predict(x1s)
    
    # Dibujamos la línea de regresión (predicción)
    plt.plot(x1s, y_pred, "k-", linewidth=2, label=r"$\hat{y}$")
    
    # Dibujamos los límites del tubo epsilon
    plt.plot(x1s, y_pred + svm_reg.epsilon, "k--")  # Límite superior
    plt.plot(x1s, y_pred - svm_reg.epsilon, "k--")  # Límite inferior
    
    # Destacamos los vectores de soporte (puntos fuera del tubo)
    plt.scatter(X[svm_reg.support_], y[svm_reg.support_], s=180, facecolors='#FFAAAA')
    
    # Dibujamos todos los puntos de datos
    plt.plot(X, y, "bo")
    
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.legend(loc="upper left", fontsize=18)
    plt.axis(axes)

# ============================================
# COMPARACIÓN VISUAL: EPSILON = 1.5 vs EPSILON = 0.5
# ============================================
plt.figure(figsize=(9, 4))

# GRÁFICO IZQUIERDO: epsilon = 1.5 (tubo ANCHO)
plt.subplot(121)
plot_svm_regression(svm_reg1, X, y, [0, 2, 3, 11])
plt.title(r"$\epsilon = {}$".format(svm_reg1.epsilon), fontsize=18)
plt.ylabel(r"$y$", fontsize=18, rotation=0)

# Dibujamos una flecha mostrando el tamaño de epsilon
plt.annotate(
        '', xy=(eps_x1, eps_y_pred), xycoords='data',
        xytext=(eps_x1, eps_y_pred - svm_reg1.epsilon),
        textcoords='data', arrowprops={'arrowstyle': '<->', 'linewidth': 1.5}
    )
plt.text(0.91, 5.6, r"$\epsilon$", fontsize=20)
# NOTA: Con tubo ANCHO, menos puntos son vectores de soporte (puntos rojos)

# GRÁFICO DERECHO: epsilon = 0.5 (tubo ESTRECHO)
plt.subplot(122)
plot_svm_regression(svm_reg2, X, y, [0, 2, 3, 11])
plt.title(r"$\epsilon = {}$".format(svm_reg2.epsilon), fontsize=18)
# NOTA: Con tubo ESTRECHO, MÁS puntos son vectores de soporte
# El modelo es más sensible a los errores pequeños

In [ ]:
# ============================================
# GENERACIÓN DE DATOS NO LINEALES PARA REGRESIÓN
# ============================================
# Ahora crearemos datos con una relación NO LINEAL (cuadrática)

np.random.seed(42)
m = 100  # 100 muestras
X = 2 * np.random.rand(m, 1) - 1  # Valores aleatorios entre -1 y 1

# Generamos datos con una relación CUADRÁTICA + ruido
# y = 0.2 + 0.1*X + 0.5*X² + ruido
y = (0.2 + 0.1 * X + 0.5 * X**2 + np.random.randn(m, 1)/10).ravel()
# NOTA: Esta es una relación NO LINEAL (parabólica)
# Un modelo lineal no podrá capturar bien esta relación

**Note**: to be future-proof, we set `gamma="scale"`, as this will be the default value in Scikit-Learn 0.22.

In [ ]:
# ============================================
# REGRESIÓN NO LINEAL CON KERNEL POLINÓMICO
# ============================================
# Para capturar la relación cuadrática, usamos SVR con kernel polinómico

from sklearn.svm import SVR

# Creamos un modelo SVR con kernel polinómico
svm_poly_reg = SVR(kernel="poly",     # Kernel polinómico (para relaciones no lineales)
                   degree=2,          # Grado 2 (cuadrático) - coincide con nuestros datos
                   C=100,             # Regularización (C alto = menos regularización)
                   epsilon=0.1,       # Ancho del tubo de tolerancia
                   gamma="scale")     # Parámetro gamma (influencia de cada punto de entrenamiento)

# Entrenamos el modelo
svm_poly_reg.fit(X, y)
# NOTA: Este modelo puede capturar relaciones NO LINEALES
# El kernel polinómico de grado 2 es perfecto para datos cuadráticos

In [ ]:
# ============================================
# COMPARACIÓN DEL PARÁMETRO C EN REGRESIÓN NO LINEAL
# ============================================

from sklearn.svm import SVR

# MODELO 1: C = 100 (Regularización DÉBIL)
# Penaliza mucho las violaciones del tubo epsilon
# El modelo se ajustará más a los datos de entrenamiento
svm_poly_reg1 = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="scale")
svm_poly_reg1.fit(X, y)

# MODELO 2: C = 0.01 (Regularización FUERTE)
# Permite más violaciones del tubo epsilon
# El modelo será más simple y generalizará mejor (pero puede ser menos preciso)
svm_poly_reg2 = SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1, gamma="scale")
svm_poly_reg2.fit(X, y)

# NOTA: El parámetro C controla el trade-off entre:
# - Simplicidad del modelo (C bajo)
# - Precisión en los datos de entrenamiento (C alto)

In [ ]:
# ============================================
# VISUALIZACIÓN FINAL: COMPARACIÓN DE C EN SVR POLINÓMICO
# ============================================

plt.figure(figsize=(9, 4))

# GRÁFICO IZQUIERDO: C=100, epsilon=0.1 (Regularización DÉBIL)
plt.subplot(121)
plot_svm_regression(svm_poly_reg1, X, y, [-1, 1, 0, 1])
plt.title(r"$degree={}, C={}, \epsilon = {}$".format(
    svm_poly_reg1.degree, 
    svm_poly_reg1.C, 
    svm_poly_reg1.epsilon), fontsize=18)
plt.ylabel(r"$y$", fontsize=18, rotation=0)
# NOTA: Con C=100, el modelo se ajusta MUY bien a los datos
# La curva sigue de cerca los puntos de entrenamiento
# Menos puntos rojos (vectores de soporte) porque el tubo se ajusta bien

# GRÁFICO DERECHO: C=0.01, epsilon=0.1 (Regularización FUERTE)
plt.subplot(122)
plot_svm_regression(svm_poly_reg2, X, y, [-1, 1, 0, 1])
plt.title(r"$degree={}, C={}, \epsilon = {}$".format(
    svm_poly_reg2.degree, 
    svm_poly_reg2.C, 
    svm_poly_reg2.epsilon), fontsize=18)
# NOTA: Con C=0.01, el modelo es MÁS SIMPLE
# La curva es menos precisa pero puede generalizar mejor
# Más puntos rojos (vectores de soporte) porque permite más violaciones
#
# CONCLUSIÓN: Elige C según tus necesidades:
# - C alto: Mayor precisión en entrenamiento (riesgo de overfitting)
# - C bajo: Mayor generalización (puede tener underfitting)